In [1]:
year = None
month = None
program = "prepilot"
study_type = "program"
mode_of_interest = "pilot_ebike"

In [2]:
SAVE_DIR = '/plots/'

In [3]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

URL not formatted, defaulting to "Stage_database"
Connecting to database URL db


In [4]:
# Settings and imports specific to this notebook

include_replaced_modes_as_valid = True # Flip this when we want to get results versus generate the replaced_mode correction graphs
model_with_sensed = False
input_dataset = "ONLY_LABELED" # "ONLY_LABELED", "ONLY_SENSED" or "BEST_AVAILABLE" for sensitivity analysis
LABEL_ASSIST_THRESHOLD = 0.3

# !apt-get update
# !apt-get install gcc -y
# !apt-get install g++ -y
# !pip install biogeme
# !pip install tqdm

import datetime
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.distributions as dist
from biogeme.expressions import Beta, DefineVariable, RandomVariable, exp, PanelLikelihoodTrajectory, bioDraws, log, MonteCarlo, Integrate
import biogeme.results as res
import pickle
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import emission.core.get_database as edb
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.storage.decorations.trip_queries as esdt
import emission.storage.decorations.timeline as esdl
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt
from uuid import UUID
import replacement_models

ModuleNotFoundError: No module named 'biogeme'

In [ ]:
# For reloading modules from Jupyter
import importlib
importlib.reload(replacement_models)

In [ ]:
# Do not run this notebook at all unless it is for a program; nbclient will run up through this cell
if study_type != "program":
    raise Exception("The plots in this notebook are only relevant to programs")

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r dic_re
%store -r dic_pur

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)

## Collect Data From Database

In [ ]:
expanded_ct, file_suffix, quality_text = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dic_re,
                                                                            dic_pur=dic_pur)

In [ ]:
# Join the expanded database data to socioeconomic data
socio_data = pd.read_csv('./Can Do Colorado eBike Program - en.csv')
socio_data.rename(columns={'Unique User ID (auto-filled, do not edit)':'user_id',
                          'Please identify which category represents your total household income, before taxes, for last year.':'HHINC',
                          'How many motor vehicles are owned, leased, or available for regular use by the people who currently live in your household?':'VEH',
                           'In which year were you born?':'AGE',
                          'Including yourself, how many people live in your home?':'HHSIZE',
                          'How many children under age 18 live in your home?':'CHILDREN',
                          'What is your gender?':'GENDER',
                          'If you were unable to use your household vehicle(s), which of the following options would be available to you to get you from place to place?':'available_modes',
                          'Are you a student?':'STUDENT'}, inplace=True)
socio_data = socio_data[~socio_data.user_id.isnull()]

# Deal with people who have multiple responses by using most recent
socio_data = socio_data.sort_values(by=['user_id', 'Timestamp'])
socio_data.drop_duplicates(subset=['user_id'], keep='last', inplace=True)
socio_data['user_id_socio'] = socio_data.user_id
socio_data = socio_data.drop(labels='user_id', axis=1)

# Lose some trips due to people with no survey responses
expanded_ct['user_id_socio'] = expanded_ct.user_id.astype(str)
expanded_ct.user_id_socio = [i.replace('-','') for i in expanded_ct.user_id_socio] # remove all dashes from strings
expanded_ct = expanded_ct.merge(socio_data, on='user_id_socio')

## Data Preprocessing

In [ ]:
# Add non-label category
expanded_ct['replaced_mode'] = expanded_ct['replaced_mode'].fillna('Unlabeled')
expanded_ct.loc[expanded_ct['replaced_mode'] == 'Unlabeled', 'Replaced_mode'] = "Unlabeled"

# Select variables of interest from complete OpenPATH data
data = expanded_ct[['Mode_confirm','Replaced_mode','replaced_mode','Trip_purpose','duration','distance_miles','start_local_dt_weekday','available_modes','AGE','HHINC','VEH','HHSIZE','CHILDREN','GENDER','STUDENT','user_id','_id','start_local_dt_year','start_local_dt_month','start_local_dt_day','cleaned_trip','start_fmt_time']].copy()

# List of variables to keep in data but not turn into categorical number variables
dont_categorize = ['user_id','_id','cleaned_trip']

# Make copy of user_id to be categorized since both versions are needed
data['user_id_int'] = data['user_id']

# Get timestamp from known year/month/day aggregated to days
data.rename(columns={'start_local_dt_year':'year','start_local_dt_month':'month','start_local_dt_day':'day'}, inplace=True)
data['date_time'] = pd.to_datetime(data[['year','month','day']])
data = data.drop(columns=['year','day'])

# Get time of day
data['hour'] = [int(x[1][:2]) for x in expanded_ct.start_fmt_time.str.split('T')]

# Fix age
data['AGE'] = 2022 - data['AGE']

# Recoded Cyclical Time of Day
hours_in_day = 24
months_in_year = 12
data['sin_time'] = np.sin(2*np.pi*data.hour/hours_in_day)
data['cos_time'] = np.cos(2*np.pi*data.hour/hours_in_day)
data['sin_month'] = np.sin(2*np.pi*data.month/months_in_year)
data['cos_month'] = np.cos(2*np.pi*data.month/months_in_year)

# Duration in minutes
data['duration'] = data['duration'] / 60

# Filter out some responses to data
data = data[~data['Mode_confirm'].isin(['Not a Trip','Other'])]
data = data[~data['Replaced_mode'].isin(['Not a Trip','Other'])]
data = data[~data['available_modes'].isin(['None', 'Prefer not to say'])]
data = data[~data['Trip_purpose'].isin(['not_a_trip','Other'])]
data = data[~data['HHINC'].isin(['Prefer not to say'])]

# Combine variable categories
data = data.replace('Gas Car, drove alone', 'car')
data = data.replace('Gas Car, with others', 's_car')
data = data.replace('Bikeshare', 's_micro')
data = data.replace('Scooter share', 's_micro')
data = data.replace('Regular Bike', 'p_micro')
data = data.replace('Skate board', 'p_micro')
data = data.replace('Train', 'transit')
data = data.replace('Free Shuttle', 'transit')
data = data.replace('Bus', 'transit')
data = data.replace('Walk', 'walk')
data = data.replace('Taxi/Uber/Lyft', 'ridehail')
data = data.replace('E-bike', 'ebike')

data['start_local_dt_weekday'] = data['start_local_dt_weekday'].replace(['1','2','3','4','5'],'1')
data['start_local_dt_weekday'] = data['start_local_dt_weekday'].replace(['0','6'],'0')

data.HHINC = data.HHINC.replace(['Less than $24,999',
                                       '$25,000-$49,999',
                                       '$50,000-$99,999',
                                       '$100,000 -$149,999',
                                       '$150,000-$199,999',
                                       '$200,000 or more'], [12500,37500,75000,125000,175000,250000])

# OHE encode categorical features (keep record)
keep = data['Trip_purpose']
data = pd.get_dummies(data, columns=['Trip_purpose'], prefix=['purp'])
data['Trip_purpose'] = keep

# keep = data['month']
# data = pd.get_dummies(data, columns=['month'], prefix=['month'])
# data['month'] = keep

# Calculate travel times for each trip, across every mode
def add_all_mode_tt(data, mode_col, duration_col, dist_col):
    mode_travel_times = {}
    for mode in pd.unique(data[mode_col]):

        # Linear model for duration based on distance for trips belonging to each mode
        mode_data = data[data[mode_col]==mode]
        regr = linear_model.LinearRegression()
        regr.fit(mode_data[dist_col].values.reshape(-1,1), mode_data[duration_col].values.reshape(-1,1))

        # Make prediction for ALL trips
        mode_duration_pred = regr.predict(data[dist_col].values.reshape(-1,1))
        mode_travel_times['tt_'+mode] = mode_duration_pred

    # Apply for each mode existing in the dataframe
    for mode in mode_travel_times:
        data[mode] = mode_travel_times[mode]

    return data

# Calculate all mode travel times and add to dataframe
data = add_all_mode_tt(data,'Mode_confirm','duration','distance_miles')

# Calculate vehicle costs based roughly on $/mi from: https://www.vtpi.org/tca/tca0501.pdf
cost_factors = {'car':0.80,
                's_car':0.40,
                'ridehail':3.00,
                's_micro':1.50,
                'transit':0.40}

def add_all_mode_cost(data, cost_factors, dist_col):
    for factor in cost_factors:
        data['cost_'+factor] = cost_factors[factor] * data[dist_col]
    return data

# Calculate all mode travel costs and add to dataframe
add_all_mode_cost(data, cost_factors, 'distance_miles')

# Normalize mode cost by income
for mode in cost_factors.keys():
    data[f"cost_{mode}"] = data[f"cost_{mode}"] / (data['HHINC'] / 1000)

# Labels for modes in the availability survey
availability_codes = {'Public transportation (bus, subway, light rail, etc.)':'transit',
                      'Get a ride from a friend or family member':'s_car',
                      'Rental car (including Zipcar/ Car2Go)':'car',
                      'Taxi (regular taxi, Uber, Lyft, etc)':'ridehail',
                      'Bicycle':'p_micro',
                      'Shared bicycle or scooter':'s_micro',
                      'Walk/roll':'walk',
                      'Skateboard':'p_micro',
                      'ebike':'ebike',
                      'None':'none'}

def add_mode_availability(data, availability_codes, availability_col, choice_col, replaced_col, is_sp):
    mode_list = np.unique(list(availability_codes.values())[:-1])
    choice_list = data[choice_col].values
    replaced_list = data[replaced_col].values
    for mode in mode_list:
        mode_avail = []
        for i, available in enumerate(data[availability_col].values):
            available_modes = [availability_codes[x] for x in available.split(';')]
            # For SP: Replacement/stated available should be 1, chosen should be 0
            if is_sp:
                if mode==choice_list[i]:
                    mode_check = False
                else:
                    mode_check = mode==replaced_list[i] or mode in available_modes
            # For RP: Chosen/replacement/stated available should be 1
            else:
                mode_check = mode==choice_list[i] or mode==replaced_list[i] or mode in available_modes
            # Keep binary list of which trips the mode was available for
            if mode_check:
                mode_avail.append(1)
            else:
                mode_avail.append(0)
        # For each mode add a column with binary availability
        data['av_'+mode] = mode_avail

    return data

# Split data into revealed choice and stated replacement choice (2 obs per trip)
data_rp = data.copy()
data_sp = data.copy()
data_rp['is_sp'] = False
data_sp['is_sp'] = True
data_rp['mode_choice'] = data_rp['Mode_confirm']
data_sp['mode_choice'] = data_sp['Replaced_mode']

# The SP data cannot include trips where the chosen/replaced modes are stated the same
# because we need to mark the chosen mode as unavailable in the SP data, which breaks the model if they're the same
data_sp = data_sp[data_sp.Mode_confirm!=data_sp.Replaced_mode]

# Make sure both chosen and replaced modes are in choice sets
data_rp = add_mode_availability(data_rp, availability_codes, 'available_modes', 'Mode_confirm', 'Replaced_mode', is_sp=False)
data_sp = add_mode_availability(data_sp, availability_codes, 'available_modes', 'Mode_confirm', 'Replaced_mode', is_sp=True)

# Combine RP/SP data. Keep RP data separate with a few additional columns for later analysis.
data = pd.concat([data_rp, data_sp])
data = data[~data['Replaced_mode'].isin(['Unlabeled', 'No Travel'])]

# Handle all variables that are ordinal; otherwise they may not end up in correct order
# Make sure that all mode variables align after being converted to numeric variables
mode_list = ['car','s_car','ridehail','transit','p_micro','s_micro','walk','ebike']
data.mode_choice = pd.Categorical(data.mode_choice, ordered=True, categories=mode_list)
data.Mode_confirm = pd.Categorical(data.Mode_confirm, ordered=True, categories=mode_list)
data.Replaced_mode = pd.Categorical(data.Replaced_mode, ordered=True, categories=mode_list)

# Convert categorical variables to numeric
cat_columns = data.select_dtypes(['object','category']).columns
cat_columns = cat_columns.drop(labels=dont_categorize)
all_categories = []
for i in range(0,len(cat_columns)):
    # Keep a record of what order the categories are in when converted
    var_categories = data[cat_columns].astype('category').iloc[:,i].cat.categories
    all_categories.append(var_categories)
data[cat_columns] = data[cat_columns].apply(lambda x: x.astype('category').cat.codes)

In [ ]:
# Show listed categories in their order
cat_code_lookup = dict(zip(cat_columns.values, [list(x.values) for x in all_categories]))
cat_code_lookup

In [ ]:
data.isna().sum()

In [ ]:
# Setting up dataframes for different analyses throughout the notebook

# Only ebike, labeled trips
df_ebike = data_rp[data_rp['Mode_confirm'].isin(['ebike'])].copy()
df_ebike = df_ebike[~df_ebike['Replaced_mode'].isin(['Unlabeled','No Travel'])]

# Only ebike, unlabeled trips
df_ebike_unlabeled = data_rp[data_rp['Mode_confirm'].isin(['ebike'])].copy()
df_ebike_unlabeled = df_ebike_unlabeled[df_ebike_unlabeled['Replaced_mode'].isin(['Unlabeled'])].copy()

# Only ebike, new trips
df_ebike_new_travel = data_rp[data_rp['Mode_confirm'].isin(['ebike'])].copy()
df_ebike_new_travel = df_ebike_new_travel[df_ebike_new_travel['Replaced_mode'].isin(['No Travel'])]

# RP data only for basic stats and analysis (Removed unlabeled and no travel)
df_rp = data[data['is_sp']==False]

# For analysis of accurately stated replacements
df_replaced_trips = data_rp[~data_rp['Replaced_mode'].isin(['No Travel','Unlabeled'])].copy()

In [ ]:
# Set up K-fold cross validation
kf = KFold(n_splits=3)

# Collect all scores to show at end of modeling
score_results = {}

In [ ]:
# Check that mode availability is being set properly
# mode_choice should always be available; in RP it is Mode_confirm in SP it is Replaced_mode
for i, mode in enumerate(cat_code_lookup['mode_choice']):
    print(mode)
    assert sum(data[data['mode_choice']==i][f"av_{mode}"]) == len(data[data['mode_choice']==i])

## Data Stats

In [ ]:
# Data stats
print(f"Trips: {len(df_rp)}")
print(f"Observed Choices: {len(data)}")
print(f"Users: {len(np.unique(data.user_id))}")
print(f"Trips per user: {len(data) / len(pd.unique(data.user_id))}")
print(f"New activity: {len(df_ebike_new_travel) / len(df_ebike)}")
print(f"Ebike all trips: {len(df_ebike)}")

# Random Forest Classifier

In [ ]:
data.columns

In [ ]:
feature_list = ['tt_ebike', 'tt_s_car', 'tt_walk', 'tt_p_micro',
       'tt_car', 'tt_transit', 'tt_s_micro', 'tt_ridehail', 'cost_car',
       'cost_s_car', 'cost_ridehail', 'cost_s_micro', 'cost_transit', 'av_car', 'av_ebike', 'av_p_micro',
       'av_ridehail', 'av_s_car', 'av_s_micro', 'av_transit', 'av_walk','purp_Entertainment/Social', 'purp_Home', 'purp_Meal',
       'purp_Personal/Medical', 'purp_Recreation/Exercise', 'purp_Religious',
       'purp_School', 'purp_Shopping', 'purp_Transit transfer', 'purp_Work','start_local_dt_weekday','sin_time', 'cos_time','sin_month','cos_month']

### Train on All Choices

In [ ]:
# Train and test model
rf, accuracy, f1, confusion = replacement_models.gbdt(data, 'mode_choice', feature_list, kf)

# Save scores for model comparison
score_results['rf_chosen_chosen'] = (np.mean(accuracy), np.mean(f1))
print(f"Accuracy: {np.mean(accuracy)}")
print(f"F1: {np.mean(f1)}")

# Average and plot the confusion matrices
confusion_mean = np.mean(np.array(confusion), axis=0)
fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(confusion_mean, annot=True, fmt='.1%', cmap='YlGnBu', linewidths=.5, xticklabels=all_categories[0].values, yticklabels=all_categories[0].values, cbar=False).set(title='Random Forest Confusion Matrix (Chosen)', xlabel='Predicted', ylabel='Actual')
plt.subplots_adjust(bottom=0.25)

rf_keep = rf

### Train on Chosen Test on Holdout Users

In [ ]:
# Train and test model (10 users for now)
accuracy_holdout = []
f1_holdout = []
confusion_holdout = []
for user in list(np.unique(df_rp.user_id))[:10]:

    # Remove user from training data, keep only user in test data
    labeled_df = data[data['user_id']!=user]
    unlabeled_df = df_rp[df_rp['user_id']==user]

    # Set the chosen mode availability to 0
    for i in range(0,len(unlabeled_df)):
        unlabeled_df[f"av_{mode_list[unlabeled_df['Mode_confirm'].iloc[i]]}"].iat[i] = 0

    # Train on all trips by other users
    rf, accuracy, f1, confusion = replacement_models.rf(labeled_df, 'mode_choice', feature_list, kf)

    # Test on the stated replacement for holdout user
    X_test = unlabeled_df[feature_list].values
    y_pred = rf.predict(X_test)
    y_test = unlabeled_df['Replaced_mode'].values

    accuracy_holdout.append(sklearn.metrics.accuracy_score(y_test, y_pred))
    f1_holdout.append(sklearn.metrics.f1_score(y_test, y_pred, average='weighted'))
    confusion_holdout.append(sklearn.metrics.confusion_matrix(y_test, y_pred, labels=[0,1,2,3,4,5,6,7], normalize='pred'))

# Save scores for model comparison
score_results['rf_holdout_replaced'] = (np.mean(accuracy_holdout), np.mean(f1_holdout))
print(f"Accuracy: {np.mean(accuracy_holdout)}")
print(f"F1: {np.mean(f1_holdout)}")

# Average and plot the confusion matrices
confusion_mean = np.mean(confusion_holdout, axis=0).reshape(8,8)
fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(confusion_mean, annot=True, fmt='.1%', cmap='YlGnBu', linewidths=.5, xticklabels=all_categories[0].values, yticklabels=all_categories[0].values, cbar=False).set(title='Random Forest Confusion Matrix (Holdout)', xlabel='Predicted', ylabel='Actual')
plt.subplots_adjust(bottom=0.25)

### Train on n Samples Chosen Test on Replaced

In [ ]:
# Train and test model
samples_min = []
user_min = []
accuracy_min = []
f1_min = []
for n_samples in range(10,200,10):

    # Sample n training data points from all users
    labeled_df = data.sample(n_samples)

    # Construct model (accuracy/f1 don't matter here; we re-test per user below)
    rf, accuracy, f1, confusion = replacement_models.rf(labeled_df, 'mode_choice', feature_list, kf)

    # Test on the stated replacement for holdout user
    for user in list(np.unique(df_rp.user_id))[:10]:
        unlabeled_df = df_rp[df_rp['user_id']==user]

        # Set the chosen mode availability to 0
        for i in range(0,len(unlabeled_df)):
            unlabeled_df[f"av_{mode_list[unlabeled_df['Mode_confirm'].iloc[i]]}"].iat[i] = 0

        # Test on the stated replacement for holdout user
        X_test = unlabeled_df[feature_list].values
        y_pred = rf.predict(X_test)
        y_test = unlabeled_df['Replaced_mode'].values

        accuracy_min.append(sklearn.metrics.accuracy_score(y_test, y_pred))
        f1_min.append(sklearn.metrics.f1_score(y_test, y_pred, average='weighted'))
        samples_min.append(n_samples)
        user_min.append(user)

In [ ]:
plot_data = pd.DataFrame({'samples_min':samples_min, 'user_min':user_min, 'accuracy_min':accuracy_min, 'f1_min':f1_min})

fig, ax = plt.subplots(figsize=(13,6))
sns.boxplot(ax=ax, data=plot_data, x='samples_min', y='accuracy_min', color='purple').set(title='Accuracy on n Samples', xlabel='n Samples', ylabel='Accuracy')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)

In [ ]:
plot_data = pd.DataFrame({'samples_min':samples_min, 'accuracy_min':accuracy_min, 'f1_min':f1_min})

fig, ax = plt.subplots(figsize=(13,6))
sns.barplot(ax=ax, data=plot_data, x='samples_min', y='accuracy_min', color='darkblue').set(title='Accuracy on n Samples', xlabel='n Samples', ylabel='Accuracy')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)

# Choice Model

## MXL

### Train on All Choices

In [ ]:
# Train and test model
mxl, accuracy, f1, confusion = replacement_models.mxl(data, 'mode_choice')

# Save scores for model comparison
score_results['mxl_chosen_chosen'] = (np.mean(accuracy), np.mean(f1))
print(f"Accuracy: {np.mean(accuracy)}")
print(f"F1: {np.mean(f1)}")

# Average and plot the confusion matrices
confusion_mean = np.mean(np.array(confusion), axis=0)
fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(confusion_mean, annot=True, fmt='.1%', cmap='YlGnBu', linewidths=.5, xticklabels=all_categories[0].values, yticklabels=all_categories[0].values, cbar=False).set(title='MXL Confusion Matrix', xlabel='Predicted', ylabel='Actual')
plt.subplots_adjust(bottom=0.25)

# Save model parameters for prediction
mxl_keep = mxl

## MNL

### Train on All Choices

In [ ]:
# Train and test model
mnl, accuracy, f1, confusion = replacement_models.mnl(data, 'mode_choice', kf)

# Save scores for model comparison
score_results['mnl_chosen_chosen'] = (np.mean(accuracy), np.mean(f1))
print(f"Accuracy: {np.mean(accuracy)}")
print(f"F1: {np.mean(f1)}")

# Average and plot the confusion matrices
confusion_mean = np.mean(np.array(confusion), axis=0)
fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(confusion_mean, annot=True, fmt='.1%', cmap='YlGnBu', linewidths=.5, xticklabels=all_categories[0].values, yticklabels=all_categories[0].values, cbar=False).set(title='MNL Confusion Matrix', xlabel='Predicted', ylabel='Actual')
plt.subplots_adjust(bottom=0.25)

mnl_keep = mnl

## Model Performance

In [ ]:
score_df = pd.DataFrame(score_results.keys(), score_results.values()).reset_index()
score_df.columns = ['Accuracy','F1','Model']
model_types = score_df['Model'].str.split('_', expand=True)
model_types.columns = ['Model Type','Train Set','Test Set']
score_df = pd.concat([score_df, model_types], axis=1)
score_df

In [ ]:
# Various model performances
plot_data = score_df[score_df['Train Set']=='chosen']
fig, ax = plt.subplots(figsize=(13,5))
sns.barplot(ax=ax, data=plot_data, x='Test Set', y='Accuracy', hue='Model Type').set(title='Model Accuracy Trained on Primary', xlabel='Model', ylabel='Accuracy')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)

In [ ]:
# Various model performances
plot_data = score_df[score_df['Train Set']=='chosen']
fig, ax = plt.subplots(figsize=(13,5))
sns.barplot(ax=ax, data=plot_data, x='Test Set', y='F1', hue='Model Type').set(title='Model F1 Trained on Primary', xlabel='Model', ylabel='F1')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)

In [ ]:
# Various model performances
plot_data = score_df
fig, ax = plt.subplots(figsize=(13,5))
sns.barplot(ax=ax, data=plot_data, x='Model', y='Accuracy').set(title='Model Accuracy', xlabel='Model', ylabel='Accuracy')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)

In [ ]:
# Various model performances
plot_data = score_df
fig, ax = plt.subplots(figsize=(13,5))
sns.barplot(ax=ax, data=plot_data, x='Model', y='F1').set(title='Model F1', xlabel='Model', ylabel='F1')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)

## Ebike Substitution Rates and Emissions

In [ ]:
# Substitution rate of ebike trips not including new trips
plot_data = df_ebike.groupby(['Replaced_mode']).count()[['Mode_confirm']].reset_index()
plot_data['subst_rate'] = plot_data['Mode_confirm'] / sum(plot_data['Mode_confirm'])

fig, ax = plt.subplots(figsize=(13,5))
sns.barplot(ax=ax, data=plot_data, x='Replaced_mode', y='subst_rate').set(title='Ebike Mode Replacement', xlabel='Replaced Mode', ylabel='Substitution Rate')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)
print(plot_data['subst_rate'])

In [ ]:
# Substitution rate of ebike trips not including new trips
# Set the chosen mode availability to 0
for i in range(0,len(df_ebike_unlabeled)):
    df_ebike_unlabeled[f"av_{df_ebike_unlabeled['Mode_confirm'].iloc[i]}"].iat[i] = 0

# Predict replaced mode for the unlabeled trips
y_pred = rf_keep.predict(df_ebike_unlabeled[feature_list].values)
df_ebike_unlabeled['Replaced_mode'] = [mode_list[y] for y in y_pred]

# Combine labeled and predicted-unlabeled ebike trips
plot_data = pd.concat([df_ebike, df_ebike_unlabeled])
plot_data = plot_data.groupby(['Replaced_mode']).count()[['Mode_confirm']].reset_index()
plot_data['subst_rate'] = plot_data['Mode_confirm'] / sum(plot_data['Mode_confirm'])

fig, ax = plt.subplots(figsize=(13,5))
sns.barplot(ax=ax, data=plot_data, x='Replaced_mode', y='subst_rate').set(title='Ebike Mode Replacement (w/Labeling)', xlabel='Replaced Mode', ylabel='Substitution Rate')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)
print(plot_data['subst_rate'])

In [ ]:
# From df_EI
# Combine variable categories
df_EI = df_EI.replace('Car, drove alone', 'car')
df_EI = df_EI.replace('Car, with others', 's_car')
df_EI = df_EI.replace('Bikeshare', 's_micro')
df_EI = df_EI.replace('Scooter share', 's_micro')
df_EI = df_EI.replace('Regular Bike', 'p_micro')
df_EI = df_EI.replace('Skate board', 'p_micro')
df_EI = df_EI.replace('Train', 'transit')
df_EI = df_EI.replace('Free Shuttle', 'transit')
df_EI = df_EI.replace('Bus', 'transit')
df_EI = df_EI.replace('Walk', 'walk')
df_EI = df_EI.replace('Taxi/Uber/Lyft', 'ridehail')
df_EI = df_EI.replace('Pilot ebike', 'ebike')
emission_rates = df_EI.groupby(['mode']).mean().reset_index()[['mode','energy_intensity_factor','CO2_factor']]
emission_rates['g_CO2_per_passmi'] = emission_rates.energy_intensity_factor*emission_rates.CO2_factor*0.000001*453.592
emission_data = plot_data.merge(emission_rates, left_on='Replaced_mode', right_on='mode')

In [ ]:
emission_data

In [ ]:
# From df_EI
emission_rates = emission_data.g_CO2_per_passmi.values
subst_rates = emission_data.subst_rate.values

# g-CO2/mi reduction through ebike availability
sum(emission_rates * subst_rates) / sum(subst_rates) - 0.007

In [ ]:
# From table in paper
emission_rates = [343.3, 18.5, 343.3, (343.3/2), 39.8, 123.8, 0.0]
subst_rates = plot_data['subst_rate'].values

# g-CO2/mi reduction through ebike availability
sum(emission_rates * subst_rates) / sum(subst_rates) - 39.8

## Explore Replaced Mode Accuracy

In [ ]:
av = {0: 'av_car',
      1: 'av_s_car',
      2: 'av_ridehail',
      3: 'av_transit',
      4: 'av_p_micro',
      5: 'av_s_micro',
      6: 'av_walk',
      7: 'av_ebike'}

In [ ]:
replaced_list = [df_replaced_trips[f"av_{x}"].iloc[i] for i, x in enumerate(df_replaced_trips.Replaced_mode)]
df_replaced_trips['replaced_in_stated'] = replaced_list

# Relabel with original mode names for plotting
for mode in av:
    mode_text = '_'.join(str(av[mode]).split('_')[1:])
    df_replaced_trips['Mode_confirm'] = df_replaced_trips['Mode_confirm'].replace(mode,mode_text)
    df_replaced_trips['Replaced_mode'] = df_replaced_trips['Replaced_mode'].replace(mode,mode_text)

df_replaced_trips['Mode_confirm'] = df_replaced_trips['Mode_confirm'].replace(7,'ebike')
df_replaced_trips['Replaced_mode'] = df_replaced_trips['Replaced_mode'].replace(7,'ebike')

In [ ]:
# Accurately stated replacement mode for all users
plot_data = df_replaced_trips[df_replaced_trips['Mode_confirm']=='ebike']
plot_data = plot_data.groupby(['date_time'], as_index=False)['replaced_in_stated'].agg(['sum','count']).apply(lambda x: x.rolling(14,1).mean())
plot_data['proportion'] = plot_data['sum'] / plot_data['count']

fig, ax = plt.subplots(figsize=(13,5))
sns.lineplot(ax=ax, data=plot_data, x='date_time', y='proportion').set(title='Proportion of Daily E-Bike Trips With Correctly Stated Replacement Mode', xlabel='Date', ylabel='Proportion Correct')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)

In [ ]:
# Accurately stated replacement mode for all users across modes
plot_data = df_replaced_trips.groupby(['Mode_confirm'], as_index=False)['replaced_in_stated'].agg(['sum','count']).reset_index()
plot_data['proportion'] = 1 - (plot_data['sum'] / plot_data['count'])

fig, ax = plt.subplots(figsize=(13,5))
sns.barplot(ax=ax, data=plot_data, x='Mode_confirm', y='proportion').set(title='Proportion of Infeasible Replacements by Primary Mode', xlabel='Primary Mode', ylabel='Proportion Incorrect')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)

In [ ]:
# Accurately stated replacement mode for all users across modes
plot_data = df_replaced_trips.groupby(['Replaced_mode'], as_index=False)['replaced_in_stated'].agg(['sum','count']).reset_index()
plot_data['proportion'] = 1 - (plot_data['sum'] / plot_data['count'])

fig, ax = plt.subplots(figsize=(13,5))
sns.barplot(ax=ax, data=plot_data, x='Replaced_mode', y='proportion').set(title='Proportion of Infeasible Replacements by Replaced Mode', xlabel='Stated Mode Replaced', ylabel='Proportion Incorrect')
plt.xticks(rotation=45)
plt.subplots_adjust(bottom=0.25)

In [ ]:
# Accurately stated replacement mode for all users
df_replaced_trips.user_id = df_replaced_trips.user_id.astype(str)
plot_data = df_replaced_trips.groupby(['user_id'], as_index=False)['replaced_in_stated'].agg(['sum','count']).reset_index()
plot_data['proportion'] = plot_data['sum'] / plot_data['count']
plot_data = plot_data.sort_values('proportion', ascending=False)

fig, ax = plt.subplots(figsize=(20,5))
sns.barplot(ax=ax, data=plot_data, x='user_id', y='proportion', color='darkblue').set(title='Proportion of Trips With Correctly Stated Replacement Mode', xlabel='User', ylabel='Proportion Correct')
plt.xticks(rotation=90)
plt.subplots_adjust(bottom=0.25)

In [ ]:
# Accurately stated replacement mode for all users
df_replaced_trips.user_id = df_replaced_trips.user_id.astype(str)
plot_data = df_replaced_trips.groupby(['user_id'], as_index=False)['replaced_in_stated'].agg(['sum','count']).reset_index()
plot_data['incorrect'] = plot_data['count'] - plot_data['sum']
plot_data['user_id'] = plot_data['user_id'].astype(str).str[-4:]
plot_data = plot_data.sort_values('incorrect', ascending=False)

fig, ax = plt.subplots(figsize=(20,5))
sns.barplot(ax=ax, data=plot_data, x='user_id', y='incorrect', color='darkblue').set(title='Trips With Unavailable Stated Replacement Mode', xlabel='User', ylabel='Count Incorrect')
plt.xticks(rotation=90)
plt.subplots_adjust(bottom=0.25)